In [1]:
from utils_aa import *

with open("params.yml", "r") as f:
    ext_params = yaml.load(f, Loader=yaml.FullLoader)
    
def objective(trial, data ,tuned_params = None, finetunning: bool = False) -> float:
    """
    """

    if  finetunning:
        seed = random.randint(1, 10_000)

        params={
                "objective":   "binary:logistic",
                "eval_metric": "logloss",
                'max_depth':   trial.suggest_int("max_depth", 2, 6, ),
                "eta":         trial.suggest_float("eta", 0.01, 0.3),
                "subsample":   trial.suggest_float("subsample", 0.5, 0.9),
                "lambda": trial.suggest_float("lambda", 0, 1),
                "alpha": trial.suggest_float("alpha",0,1),
                "scale_pos_weight": trial.suggest_float("scale_pos_weight",0,2)
            }
    else:
        seed = trial.suggest_int("seed", 1, 10_000)
        params = tuned_params

    model_instance = generate_model(
        "HOMA-IR alterado",
        data, #
        removed_features=ext_params["feature_engineering"]["removed_features"],
        xg_params=params,
        kfold_splits=5,
        seed=seed,
    )

    return (
        model_instance.get_AUC_on_test_data(),
        model_instance.get_feature_metrics()[ext_params['metrics'][0]][ext_params['metrics'][1]],
    )
    
    
    
hyperparameters_fine_tuning = optuna.create_study(
    directions=ext_params["directions"],
    sampler=optuna.samplers.NSGAIISampler()
    )

hyperparameters_fine_tuning.optimize(lambda trial: objective(trial, "data/data.csv", 
                                                             finetunning = True), n_trials=5000, n_jobs=-1)


[I 2023-05-06 18:58:39,202] A new study created in memory with name: no-name-01389666-5081-42a6-923b-2b2366e0f9f8
[I 2023-05-06 18:58:41,416] Trial 4 finished with values: [0.5, 0.0] and parameters: {'max_depth': 3, 'eta': 0.2785894250294759, 'subsample': 0.7347274549203099, 'lambda': 0.15520533667961622, 'alpha': 0.635388942300964, 'scale_pos_weight': 0.15715911751738343}. 
[I 2023-05-06 18:58:41,447] Trial 2 finished with values: [0.5, 0.0] and parameters: {'max_depth': 2, 'eta': 0.14845292988111927, 'subsample': 0.5962795065442072, 'lambda': 0.22207879660455743, 'alpha': 0.11368186604026276, 'scale_pos_weight': 0.057366185101132894}. 
[I 2023-05-06 18:58:41,526] Trial 14 finished with values: [0.8846153846153846, 0.0] and parameters: {'max_depth': 4, 'eta': 0.29036226500052836, 'subsample': 0.7926111101803128, 'lambda': 0.5509426011958966, 'alpha': 0.32168097742457036, 'scale_pos_weight': 0.7699860237970175}. 
[I 2023-05-06 18:58:41,564] Trial 5 finished with values: [0.788461538461

In [5]:
models_sampling = optuna.create_study(
    directions=ext_params["directions"],
    sampler=optuna.samplers.NSGAIISampler()
    )



models_sampling.optimize(lambda trial: objective(trial, "data/data.csv",  tuned_params = get_median_dict(hyperparameters_fine_tuning),
                                                 finetunning = False), n_trials=5000, n_jobs=-1)

make_pareto_plot(models_sampling)
make_ranking_plots(
    models_sampling, get_median_dict(hyperparameters_fine_tuning), data ="data/data.csv", CUTOFF_AUC =.9, CUTOFF_PHE=.0)


[I 2023-05-06 19:04:35,989] A new study created in memory with name: no-name-f58583cc-fa1f-4900-b08b-53b46880d450
[I 2023-05-06 19:04:39,622] Trial 2 finished with values: [0.9722222222222222, 0.0] and parameters: {'seed': 1729}. 
[I 2023-05-06 19:04:39,641] Trial 14 finished with values: [1.0, 0.0] and parameters: {'seed': 9119}. 
[I 2023-05-06 19:04:39,763] Trial 11 finished with values: [0.75, 0.0] and parameters: {'seed': 8468}. 
[I 2023-05-06 19:04:40,045] Trial 0 finished with values: [0.5138888888888888, 0.0] and parameters: {'seed': 7681}. 
[I 2023-05-06 19:04:40,077] Trial 6 finished with values: [0.5, 0.0] and parameters: {'seed': 241}. 
[I 2023-05-06 19:04:40,218] Trial 10 finished with values: [0.6923076923076923, 0.0] and parameters: {'seed': 2178}. 
[I 2023-05-06 19:04:40,383] Trial 1 finished with values: [0.9444444444444445, 0.0] and parameters: {'seed': 2344}. 
[I 2023-05-06 19:04:40,627] Trial 5 finished with values: [0.8194444444444445, 0.0] and parameters: {'seed': 

KeyboardInterrupt: 